In [1]:
from typing import List, Tuple, Dict
from copy import copy

from tqdm import tqdm_notebook
import numpy as np
import pandas as pd

In [2]:
b_test = pd.read_csv('b_train.csv',header=None)
c_test = pd.read_csv('c_train.csv',header=None)

In [3]:
c_test = c_test.transpose() 
b_test = b_test.transpose() 

In [4]:
c_test = c_test.values
b_test = b_test.values

In [5]:
c_test.shape, b_test.shape

((11, 96000), (11, 96000))

In [6]:
96000 / 8000

12.0

In [7]:
test_backdoor = b_test[:-1, :]
test_clean = c_test[:-1, :]

In [8]:
test_backdoor.shape, test_clean.shape

((10, 96000), (10, 96000))

In [9]:
96000 / 24

4000.0

In [10]:
sub_matrices = np.split(test_clean, 24, axis=1)

# 将子矩阵展开成 15 个 (1, 40000) 的矩阵
flat_matrices = [sub_matrix.reshape(1, -1) for sub_matrix in sub_matrices]

# 合并展开后的矩阵为一个形状为 (15, 40000) 的矩阵
test_c = np.concatenate(flat_matrices)

print(test_c.shape)  # 输出 (15, 40000)

(24, 40000)


In [11]:
sub_matrices = np.split(test_backdoor, 24, axis=1)

# 将子矩阵展开成 15 个 (1, 40000) 的矩阵
flat_matrices = [sub_matrix.reshape(1, -1) for sub_matrix in sub_matrices]

# 合并展开后的矩阵为一个形状为 (15, 40000) 的矩阵
test_b = np.concatenate(flat_matrices)

print(test_b.shape)  # 输出 (15, 40000)

(24, 40000)


In [12]:
res_clean  = np.concatenate((test_c,  np.ones((len(test_c[: ,-1]), 1))), axis=1)

In [13]:
res_backdoor  = np.concatenate((test_b,  np.zeros((len(test_b[: ,-1]), 1))), axis=1)

In [14]:
res_clean.shape, res_backdoor.shape

((24, 40001), (24, 40001))

In [15]:
model_results = np.vstack((res_clean, res_backdoor))

In [16]:
model_results.shape

(48, 40001)

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
model_label = model_results[:, -1]

In [19]:
model = model_results[:, :-1]

In [20]:
forest = RandomForestClassifier(n_estimators = 100000)

In [21]:
forest_fit = forest.fit(model,model_label)

In [31]:
b = pd.read_csv('./final-csv/b_train_test.csv',header=None)
c = pd.read_csv('./final-csv/c_train_test.csv',header=None)

In [32]:
c = c.transpose() 
b = b.transpose() 

In [33]:
c.shape, b.shape

((11, 79800), (11, 79800))

In [34]:
c = c.values 
b = b.values

In [35]:
test_backdoor = b[:-1, :]
test_clean = c[:-1, :]

In [26]:
c.shape

(11, 79800)

In [27]:
sub_matrices = np.split(test_clean, 10, axis=1)

# 将子矩阵展开成 15 个 (1, 40000) 的矩阵
flat_matrices = [sub_matrix.reshape(1, -1) for sub_matrix in sub_matrices]

# 合并展开后的矩阵为一个形状为 (15, 40000) 的矩阵
c = np.concatenate(flat_matrices)

print(c.shape)  # 输出 (15, 40000)

(10, 79800)


In [28]:
sub_matrices = np.split(test_backdoor, 10, axis=1)

# 将子矩阵展开成 15 个 (1, 40000) 的矩阵
flat_matrices = [sub_matrix.reshape(1, -1) for sub_matrix in sub_matrices]

# 合并展开后的矩阵为一个形状为 (15, 40000) 的矩阵
b = np.concatenate(flat_matrices)

print(b.shape)  # 输出 (15, 40000)

(10, 79800)


In [29]:
mem_res_clean = forest.predict(c)
mem_res_backdoor = forest.predict(b)

ValueError: X has 79800 features, but RandomForestClassifier is expecting 40000 features as input.

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

In [ ]:
y_pred = np.hstack((mem_res_clean, mem_res_backdoor))
y_true = np.concatenate((np.ones_like(mem_res_clean), np.zeros_like(mem_res_backdoor)))

In [ ]:
accuracy_score(y_true, y_pred)

In [ ]:
precision_score(y_true, y_pred)

In [ ]:
f1_score(y_true, y_pred)

In [ ]:
y_true

In [ ]:
y_pred